# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [180]:
import pandas as pd
import numpy as np
import math
import json
import re
from sklearn.preprocessing import MultiLabelBinarizer
%matplotlib inline

In [162]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [21]:
print(portfolio.shape)
portfolio.head()

(10, 6)


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [56]:
print('offer types:', portfolio.offer_type.unique())
print('difficulty levels:', portfolio.difficulty.unique())
print('rewards :',portfolio.reward.unique())
print('duration of offers:', portfolio.duration.unique())
allchannles = []
for val in portfolio.channels:
    allchannles.extend(set(val)-set(allchannles))
print('All cahnnels:', allchannles)

offer types: ['bogo' 'informational' 'discount']
difficulty levels: [10  0  5 20  7]
rewards : [10  0  5  3  2]
duration of offers: [ 7  5  4 10  3]
All cahnnels: ['email', 'mobile', 'social', 'web']


**Steps to prepare portfolio data:<br>
1.Renaming id as offerid.<br>
2.one hot encode channels column and remove the original column.<br>
3.One hot encode offer_type and remove the original column.**

In [142]:
def prepare_portfolio(df):
    '''
    INPUT:
        df:portfolio dataframe.
    OUTPUT:
        clean_portfolio: Cleaned portfolio data.
    '''
    df.rename(columns={'id':'offerid'}, inplace=True)
    offertypes = pd.get_dummies(df.offer_type)
    channel_binarizer = MultiLabelBinarizer()
    channel_df = pd.DataFrame(channel_binarizer.fit_transform(df.channels), columns=channel_binarizer.classes_)
    
    clean_portfolio = pd.concat([df, offertypes, channel_df], axis=1)
    clean_portfolio.drop(['offer_type', 'channels'], inplace=True, axis=1)
    
    return clean_portfolio

In [53]:
clean_portfolio = prepare_portfolio(portfolio)
clean_portfolio

,reward,difficulty,duration,id,bogo,discount,informational,email,mobile,social,web
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,1,1,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0,1
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0,1
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0,1
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1,1,1,0
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0,1


In [12]:
print(profile.shape)
profile.head()

(17000, 5)


,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [63]:
print('Gender values:',profile.gender.unique())
print('Age values:', profile.age.unique())

Gender values: [None 'F' 'M' 'O']
M    8484
F    6129
O     212
Name: gender, dtype: int64
Age values: [118  55  75  68  65  58  61  26  62  49  57  40  64  78  42  56  33  46
  59  67  53  22  96  69  20  45  54  39  41  79  66  29  44  63  36  76
  77  30  51  27  73  74  70  89  50  90  60  19  72  52  18  71  83  43
  47  32  38  34  85  48  35  82  21  24  81  25  37  23 100  28  84  80
  87  86  94  31  88  95  93  91  92  98 101  97  99]


In [66]:
den =100/profile.shape[0]
print(profile['gender'].value_counts() * den)
print('\n Percentage of missing values:')
profile.isnull().sum()*den

M    49.905882
F    36.052941
O     1.247059
Name: gender, dtype: float64

 Percentage of missing values:


gender              12.794118
age                  0.000000
id                   0.000000
became_member_on     0.000000
income              12.794118
dtype: float64

In [88]:
print('Missing values count :', profile.isnull().sum())
profile.age.value_counts()

Missing values count : gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64


118    2175
58      408
53      372
51      363
54      359
       ... 
100      12
96        8
98        5
99        5
101       5
Name: age, Length: 85, dtype: int64

**It appears that there is some relation between the missing values and age=118. Lets confirm this.**

In [92]:
profile118 = profile[profile['age'] == 118]
print('Missing values with age=118:\n',profile.isnull().sum())
profile118.head()

Missing values with age=118:
 gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64


,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
6,None,118,8ec6ce2a7e7949b1bf142def7d0e0586,20170925,NaN
7,None,118,68617ca6246f4fbc85e91a2a49552598,20171002,NaN


**Well Well, all the 2175 of the missing values are having an age=118. This makes it simpler for us to drop these rows. As gender and income both are missing we don't need to think anymore before dropping them.**

**Steps to prepare profile:<br>
1.Drop all the rows with age=118.<br>
2.Convert became_member_on to datetime.<br>
3.Extract year and create dummies.<br>
4.Create gender dummies<br>
5.Rename id to person**

In [218]:
def prepare_profile(df):
    '''
    INPUT:
        df: Profile dataframe
    OUTPUT:
        cleaned_profile: Cleaned profile dataframe
    '''
    df.rename(columns={'id':'person'}, inplace=True)
    drop118 = df[df['age'] ==118].index
    df = df.drop(index= drop118, axis=0).reset_index(drop=True)
    df['date'] = pd.to_datetime(df['became_member_on'], format='%Y%m%d')
    
#     df_gender_O = df[df['gender'] == 'O'].index
#     df = df.drop(index=df_gender_O, axis=0).reset_index(drop = True)
    gender = pd.get_dummies(df.gender, prefix='gender')
    
    membership_year = df.date.dt.year
    membership_dummies = pd.get_dummies(membership_year)
    
    cleaned_profile = pd.concat([df, membership_dummies,gender], axis =1)
    #cleaned_profile['gender'] = cleaned_profile['gender'].apply(lambda x: 1 if x=='M' else 0)
    cleaned_profile.drop(['became_member_on','gender'], inplace=True, axis=1)
    
    return cleaned_profile    

In [219]:
clean_profile = prepare_profile(profile)
clean_profile.head()

,age,person,income,date,2013,2014,2015,2016,2017,2018,gender_F,gender_M,gender_O
0,55,0610b486422d4921ae7d2bf64640c50b,112000.0,2017-07-15,0,0,0,0,1,0,1,0,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2017-05-09,0,0,0,0,1,0,1,0,0
2,68,e2127556f4f64592b11af22de27a7932,70000.0,2018-04-26,0,0,0,0,0,1,0,1,0
3,65,389bc3fa690240e798340f5a15918d5c,53000.0,2018-02-09,0,0,0,0,0,1,0,1,0
4,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2017-11-11,0,0,0,0,1,0,0,1,0


In [37]:
print(transcript.shape)
transcript.head()

(306534, 4)


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [178]:
print('Number of Unique Users: ', transcript.person.nunique())
print('Event unique values:',transcript.event.unique())
print('Time unique values:',transcript.time.unique())

Number of Unique Users:  17000
Event unique values: ['offer received' 'offer viewed' 'transaction' 'offer completed']
Time unique values: [  0   6  12  18  24  30  36  42  48  54  60  66  72  78  84  90  96 102
 108 114 120 126 132 138 144 150 156 162 168 174 180 186 192 198 204 210
 216 222 228 234 240 246 252 258 264 270 276 282 288 294 300 306 312 318
 324 330 336 342 348 354 360 366 372 378 384 390 396 402 408 414 420 426
 432 438 444 450 456 462 468 474 480 486 492 498 504 510 516 522 528 534
 540 546 552 558 564 570 576 582 588 594 600 606 612 618 624 630 636 642
 648 654 660 666 672 678 684 690 696 702 708 714]


In [179]:
print('Null values in the transcript data:')
transcript.isnull().sum()

Null values in the transcript data:


person    0
event     0
value     0
time      0
dtype: int64

In [229]:
transcript['time'] /=24.0

As provided in the description the 'value' column contains either an offer id or transcation amount. Lets transform the column and see what it contains.

In [204]:
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

It appears that there are four different 'keys' used in the dictionary namely: offer id, offer_id, amount, reward.

From the above cell we can see that some offerids are distributed in both the columns. We need to combine these two columns together into one, this might reduce the number of missing values.
Summing up these above values, it can be said that the column 'offer_id' is related to 'reward'(completed offers).The amount column is surely a 'transaction' event.'offer id' is sum of the 'offer received' and 'offer viewed' events.

In [230]:
customers = clean_profile.person
customers.shape

(14825,)

In [231]:
transcript_cust = transcript[transcript['person'].isin(customers)]

In [232]:
transcript_cust.person.nunique()

14825

In [245]:
transactions = transcript_cust[transcript_cust['event']=='transaction'].reset_index(drop=True)
transactions['amount']=transactions['value'].apply(lambda x: list(x.values())[0])
transactions.drop(columns =['event','value'], axis=1, inplace=True)
transactions.head()

,person,time,amount
0,02c083884c7d45b39cc68e1314fec56c,0.0,0.83
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0.0,34.56
2,54890f68699049c2a04d415abc25e717,0.0,13.23
3,b2f1cd155b864803ad8334cdf13c4bd2,0.0,19.51
4,fe97aa22dd3e48c8b143116a8403dd52,0.0,18.97


In [256]:
offers = transcript_cust[transcript_cust['event']!='transaction'].reset_index(drop=True)
event = offers.event.apply(lambda x : re.match('^offer ([a-z]+$)',x).groups(1)[0])
event = pd.get_dummies(event)
offers['offer_id']=offers['value'].apply(lambda x: list(x.values())[0])
offers = pd.concat([offers,event],axis=1)
offers.drop(columns =['event','value'],inplace=True)
offers.head()

,person,time,offer_id,completed,received,viewed
0,78afa995795e4d85b5d9ceeca43f5fef,0.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,0
1,e2127556f4f64592b11af22de27a7932,0.0,2906b810c7d4411798c6938adc9daaa5,0,1,0
2,389bc3fa690240e798340f5a15918d5c,0.0,f19421c1d4aa40978ebb69ca19b0e20d,0,1,0
3,2eeac8d8feae4a8cad5a6af0499a211d,0.0,3f207df678b143eea3cee63160fa8bed,0,1,0
4,aa4862eba776480b8bb9c68455b8c2e1,0.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0


,person,time,offer_id,completed,received,viewed
0,78afa995795e4d85b5d9ceeca43f5fef,0.00,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,1,0
1,e2127556f4f64592b11af22de27a7932,0.00,2906b810c7d4411798c6938adc9daaa5,0,1,0
2,389bc3fa690240e798340f5a15918d5c,0.00,f19421c1d4aa40978ebb69ca19b0e20d,0,1,0
3,2eeac8d8feae4a8cad5a6af0499a211d,0.00,3f207df678b143eea3cee63160fa8bed,0,1,0
4,aa4862eba776480b8bb9c68455b8c2e1,0.00,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0
...,...,...,...,...,...,...
148800,84fb57a7fe8045a8bf6236738ee73a0f,29.75,5a8bc65990b245e5a138643cd4eb9837,0,0,1
148801,abc4359eb34e4e2ca2349da2ddf771b6,29.75,3f207df678b143eea3cee63160fa8bed,0,0,1
148802,8dda575c2a1d44b9ac8e8b07b93d1f8e,29.75,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,0,1
148803,8431c16f8e1d440880db371a68f82dd0,29.75,fafdcd668e3743c1bb461111dcafc2a4,1,0,0
